In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
from sklearn.datasets import load_iris
data = load_iris()
# data

## 构建训练数据集与搭建网络

In [4]:
x = data['data']
y = data['target']
# 转化数据类型
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)
# 构建数据集
from torch.utils.data import Dataset, DataLoader
class IrisDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

# 切分数据集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# 构建数据集和数据加载器
train_dataset = IrisDataset(x_train, y_train)
test_dataset = IrisDataset(x_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [5]:
# 构建一个3层nn
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 训练模型并测试模型准确率

In [6]:
# 训练模型
def train_model():
    # 训练模型
    net = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
    loss_stat = []
    for epoch in range(100):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / 10
        # print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, avg_loss))
        loss_stat.append(avg_loss)
        running_loss = 0.0
    # print('Finished Training')
    return net, loss_stat

In [7]:
# 评估模型
def test_model(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            y = net(inputs)
            _, predicted = torch.max(y.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 100 test images: %d %%' % (100 * correct / total))
    return correct, total
        

In [8]:
model_file_name = 'iris_net.pth'
net = None

if model_file_name in os.listdir():
    net = Net()
    net.load_state_dict(torch.load(model_file_name, weights_only=True))
else:
    net, loss_stat = train_model()
    torch.save(net.state_dict(), model_file_name)
    plt.plot(loss_stat)
    
test_model(net)

Accuracy of the network on the 100 test images: 96 %


(29, 30)

## 模型转换以及输出测试
将Pytorch模型转化位ONNX模型并对比输出的结果

In [9]:
# 保存模型为onnx
torch.onnx.export(net, torch.randn(1, 4), 'iris_net.onnx', input_names=['input'], output_names=['output'])

In [10]:
x_test[:1]

tensor([[6.1000, 2.8000, 4.7000, 1.2000]])

In [11]:
# 加载onnx模型并执行推理，对比torch模型的输出与onnx的输出的差异
import onnxruntime as ort
ort_session = ort.InferenceSession('iris_net.onnx')
ort_inputs = {'input': x_test[:1].numpy()}
onnx_result = ort_session.run(['output'], ort_inputs)
onnx_result

[array([[-3.4283571,  2.8357146, -0.2446419]], dtype=float32)]

In [12]:
torch_result = net(x_test[:1])
torch_result

tensor([[-3.4284,  2.8357, -0.2446]], grad_fn=<AddmmBackward0>)

## 量化测试过程

In [13]:
import sys
sys.path.append('../..')
# import scripts.nnom as nnom
import scripts.nnom_onnx_utils as utils

In [26]:
# 随机生成一些变量
x = np.random.randn(1,10) * 5
x[0][0] = 32.0
dec = utils.find_dec_bits_max_min(x)
qx = utils.quantize_data(x, dec).astype(dtype=np.int8)
cx = qx * 2**(-dec)
dx = x - cx
print('原始数据: ', x)
print('量化数据:', qx)
print('dec:', dec)
print('还原后：', cx)
print('误差：', dx)

原始数据:  [[32.         -3.95491664 -4.37047222 -8.03214777 -3.42550685 -6.389711
  -4.84653968  7.16511194  9.58964002 -0.52632722]]
量化数据: [[127 -16 -17 -32 -14 -26 -19  29  38  -2]]
dec: 2
还原后： [[31.75 -4.   -4.25 -8.   -3.5  -6.5  -4.75  7.25  9.5  -0.5 ]]
误差： [[ 0.25        0.04508336 -0.12047222 -0.03214777  0.07449315  0.110289
  -0.09653968 -0.08488806  0.08964002 -0.02632722]]


In [27]:
# 读取onnx模型的参数
import onnx
onnx_model = onnx.load('iris_net.onnx')
onnx.checker.check_model(onnx_model)

# Get input ，Node and output name
input_name = onnx_model.graph.input[0].name # currently only support one input
output_name = onnx_model.graph.output[0].name # currently only support one output
print('input_name: ', input_name)
for node in onnx_model.graph.node:
    print(node.op_type, node.input, node.output)

input_name:  input
Gemm ['input', 'fc1.weight', 'fc1.bias'] ['/fc1/Gemm_output_0']
Relu ['/fc1/Gemm_output_0'] ['/Relu_output_0']
Gemm ['/Relu_output_0', 'fc2.weight', 'fc2.bias'] ['/fc2/Gemm_output_0']
Relu ['/fc2/Gemm_output_0'] ['/Relu_1_output_0']
Gemm ['/Relu_1_output_0', 'fc3.weight', 'fc3.bias'] ['output']
